Infrared Range Sensor
===============
<img src="files/img/gp2d120_output.png" align="right">
<img src="files/img/gp2d12.jpg" align="left">
The Infrared Range Sensor enables your robot to detect objects. Unlike ultrasonic sensors, which use sound waves to measure distance, the IR Range Sensor uses infrared light. On one side of the sensor an infrared LED shines a beam of light, which is reflected back by the closest object, and detected by the receiver on the other side of the sensor. The sensor then uses a method called triangulation to determine how far away the object was.
<p>The basis for triangulation is that objects at different distances will reflect the infrared beam back to the receiver at different angles. The varying angles produce different voltage levels in the sensor, and in turn sensor values that can be used to calculate distance. See below:</p>
<p align="center"><img src="files/img/triangulation.png"></p>
<p>The IR Range Sensor provides very reliable distance values ranging from 4 to 30 centimeters away. One big advantage with using the IR Range Sensor is that it's not affected by soft or angled objects that cause the ultrasonic sensor to fail.</p>
<p>One challenge with using the sensor is that the raw values provided do not directly correlate to useful distance values, and they're also non-linear (see figure at the right). This means that we must perform a calculation on the raw sensor data first.</p>
<p>One additional challenge is that we can't directly access the voltage returned by the sensor in our program. What we can access is the sensor value, which is proportional to the voltage - we'll just have to take a conversion factor into account.</p>

First, let's initialize the IR Range sensor that is connected to port S4 in the BrickPi.

In [ ]:
from BrickPi import *
BrickPiSetup()
BrickPi.SensorType[PORT_4] = TYPE_SENSOR_RAW
BrickPiSetupSensors()

Now, let's make one measurement; the resulting number is proportional to the voltage signal, which in turn is related to the distance to an obstacle in the front, according to the graph presented before.

In [ ]:
BrickPiUpdateValues()
V = BrickPi.Sensor[PORT_4]
print(V)

If no obstacle is detected, a low value around 137 is returned.

The maximum value, around 755, is returned when an object is very close to the sensor, 2-3 cm.

If the value is 1023, something wrong happened. Please try again, or ask the teacher.


Calibration
===========
During the testing, you will notice that the sensors’ outputs are not in perfect accordance with the
measuring characteristics graphs. Even two pieces of the same type of sensor return different values
for the same actual obstacle distance. Thus it is necessary to perform some calibration.

A good method is to simply make a table of points (sensor, distance) and perform a simple linear interpolation
between those points. 

<center>
<table>
    <tr>
        <th>Sensor value</th><th>Distance</th>
    </tr>
    <tr>
        <td>&nbsp;</td><td style="text-align:center">3</td>
    </tr>
    <tr>
        <td>&nbsp;</td><td style="text-align:center">4</td>
    </tr>
    <tr>
        <td>&nbsp;</td><td style="text-align:center">8</td>
    </tr>
    <tr>
        <td>&nbsp;</td><td style="text-align:center">12</td>
    </tr>
    <tr>
        <td>&nbsp;</td><td style="text-align:center">16</td>
    </tr>
    <tr>
        <td>&nbsp;</td><td style="text-align:center">20</td>
    </tr>
    <tr>
        <td>&nbsp;</td><td style="text-align:center">24</td>
    </tr>
    <tr>
        <td>&nbsp;</td><td style="text-align:center">28</td>
    </tr>
</table>
</center>

You should make the following experiment:
<ul>
    <li>Place your robot on a mat with a distance scale</li>
    <li>Put a box in front of the robot sensor at a given distance according to the scale</li>
    <li>Read the sensor value and write the result in the table</li>
    <li>Repeat the procedure for all the distances in the table</li>
</ul>

In [ ]:
BrickPiUpdateValues()
V = BrickPi.Sensor[PORT_4]
print(V)

You can verify the measured values by plotting the table in a graph with the following code. Just add the measured values into vector $S$. You should get a set of points placed like the graph described in the sensor datasheet.

In [ ]:
%matplotlib inline
from matplotlib.pyplot import *

S = [760,680,445,350,297,266,248,230]
D = [3, 4, 8, 12, 16, 20, 24, 38]
plot(D,S,'*');
xlabel('Distance (cm)');
ylabel('Sensor value');

Now, let's define a function for linear interpolation.

In [ ]:
def interpolateDistance(V):
    i = 0
    while i < len(S) and S[i] > V:
        i = i + 1
    if i==0:
        d = float(D[0])
    elif i==len(S):
        d = float(D[-1])
    else:
        SI = S[i-1]
        SF = S[i]
        DI = D[i-1]
        DF = D[i]
        d = DI + float(DF-DI)*(V-SI)/(SF-SI)
    return d

And let's plot it in the sensor range:

In [ ]:
V = range(230,760)
d = []
for i in V:
    d.append(interpolateDistance(i))
plot(d,V);

For visualization purposes, let's define a short function for plotting the sensor image and the distance as a triangle.

In [ ]:
ir_img = imread('img/sharp.png')

def drawInfrared(d):
    imshow(ir_img)
    hold(True)
    plot([48,48+8*d,48+8*d,48],[60,60+d/4,60-d/4,60])
    axis([0,400,0,120])
    axis('off')
    hold(False)

Let's visualize a measurement. Put an obstacle (your hand, a book) in front of the robot and refresh the plot by pressing Ctrl+Enter.

In [ ]:
BrickPiUpdateValues()
V = BrickPi.Sensor[PORT_4] 
d = interpolateDistance(V)
drawInfrared(d)
print((V,d))

We can add a few more lines for repeating automatically the measurement process (this is called a **loop** in programming).

For stopping the loop, you should click the **Interrupt** button in the notebook menu.

![](files/img/interrupt_button.png)

In [ ]:
from IPython.display import display, clear_output
try:
    while True:
        BrickPiUpdateValues()
        V = BrickPi.Sensor[PORT_4]
        d = interpolateDistance(V)
        drawInfrared(d)
        display(gcf())
        clear_output(wait=True)
        time.sleep(1)
except KeyboardInterrupt:
    clf();

Congratulations! You have successfully learnt about the IR range sensors. Now you are ready to program the robot for some useful tasks with the sensor.
<hr>
<center>
    <p>&copy; Enric Cervera 2017</p>
    <p><a href="http://www.uji.es"><img alt="Universitat Jaume I" src="img/uji_new_logo.png" /></a></p>
    <p><a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.</p>
</center>